In [1]:
# Amazon Repatriation Automation
## Jamie needs to concatenate sales data received from markets into one excel file
#### Concatenate ICLA and Classical Sheet into one excel
##### Jamie needs to send over Cost Feed from Amazon, Concatenated Sales Feed from markets, ICLA exceptions / Classical Sheet and International Catalogue
### remove all currency symbols from cost data
###### create dashboard from the data

In [2]:
#importing libraries

import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from google.cloud import bigquery


In [3]:
#importing sales data

sales = pd.read_excel('G:\Amazon Repatriation May 2020\Amazon Repat - Amazon Data April.xlsx', dtype={'ean': str})

In [4]:
#importing cost data

costs = pd.read_excel('G:\Amazon Repatriation May 2020\Amazon Cost Feed Combined - April.xlsx', dtype={'UPC': str })

In [5]:
### reduce table size 
costs2 = costs[['Region','UPC','Transfer Price','Dealer Price','Net Unit Price','Currency ']]

In [6]:
### amount of rows in cost data
costs2.shape

(83531, 6)

In [7]:
#keeping any rows with Transfer Price below .30 euros
costs3 = costs2[costs2['Transfer Price'] >= .3] 

In [8]:
### amount of rows in cost data
costs3.shape

(55654, 6)

In [9]:
###check for duplicates rows
sum(costs3.duplicated())

864

In [10]:
##drop duplicates
costs4 = costs3.drop_duplicates(ignore_index = True)

In [11]:
###check for duplicates rows
sum(costs4.duplicated())

0

In [12]:
sales.shape

(335395, 10)

In [13]:
#concatenating inventrory owner groups, vendor groups for streamlined data

sales.loc[sales['inventory_owner_group'] == 'UK', 'inventory_owner_group_combined'] = 'UK'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.co.uk', 'inventory_owner_group_combined'] = 'UK'
sales.loc[sales['inventory_owner_group'] == 'DE', 'inventory_owner_group_combined'] = 'DE'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.de', 'inventory_owner_group_combined'] = 'DE'
sales.loc[sales['inventory_owner_group'] == 'IT', 'inventory_owner_group_combined'] = 'IT'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.it', 'inventory_owner_group_combined'] = 'IT'
sales.loc[sales['inventory_owner_group'] == 'FR', 'inventory_owner_group_combined'] = 'FR'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.fr', 'inventory_owner_group_combined'] = 'FR'
sales.loc[sales['inventory_owner_group'] == 'ES', 'inventory_owner_group_combined'] = 'ES'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.es', 'inventory_owner_group_combined'] = 'ES'
sales.loc[sales['inventory_owner_group'] == 'Pan-EU SCO', 'inventory_owner_group_combined'] = 'Pan-EU SCO'
sales.loc[sales['vendor_group'] == 'UMG UK', 'vendor_group2'] = 'UK'
sales.loc[sales['vendor_group'] == 'UMG DE', 'vendor_group2'] = 'DE'
sales.loc[sales['vendor_group'] == 'UMG FR', 'vendor_group2'] = 'FR'
sales.loc[sales['vendor_group'] == 'UMG IT', 'vendor_group2']  ='IT'
sales.loc[sales['vendor_group'] == 'UMG ES', 'vendor_group2'] ='ES'
sales.loc[sales['vendor_group'] == 'UMG AT', 'vendor_group2'] ='AT'
sales.loc[sales['vendor_group'] == '3rd Party', 'vendor_group2'] = '3rd Party'

In [14]:
#creating movement key

sales['movement'] = sales['vendor_group'] + ' to ' +sales['inventory_owner_group_combined'] 

In [15]:
sales.shape

(335395, 13)

In [16]:
#splitting up strings of UPC Amazon delivers to us

new = sales['ean'].str.split("/", n =-1, expand = True) 
sales["ean1"]= new[0] 
sales["ean2"]= new[1] 
sales["ean3"]= new[2] 
sales["ean4"]= new[3] 
sales["ean5"]= new[4] 
sales.drop(columns =["ean"], inplace = True) 

In [22]:
sales['ean1'] = sales['ean1'].str.lstrip('0')

In [23]:
sales['ean2'] = sales['ean2'].str.lstrip(' 0')

In [24]:
sales['ean3'] = sales['ean3'].str.lstrip(' 0')

In [25]:
sales['ean4'] = sales['ean4'].str.lstrip(' 0')

In [26]:
sales['ean5'] = sales['ean5'].str.lstrip(' 0')

In [28]:
### merge data
combo1 = sales.merge(costs4, how='inner', left_on=["ean1","vendor_group2"], right_on=["UPC","Region"])

In [29]:
### merge data
combo2 = sales.merge(costs4, how='inner', left_on=["ean2","vendor_group2"], right_on=["UPC","Region"])

In [30]:
### merge data
combo3 = sales.merge(costs4, how='inner', left_on=["ean3","vendor_group2"], right_on=["UPC","Region"])

In [31]:
### merge data
combo4 = sales.merge(costs4, how='inner', left_on=["ean4","vendor_group2"], right_on=["UPC","Region"])

In [32]:
### merge data
combo5 = sales.merge(costs4, how='inner', left_on=["ean5","vendor_group2"], right_on=["UPC","Region"])

In [33]:
frames = [combo1,combo2,combo3,combo4,combo4]

combo6 = pd.concat(frames)

In [35]:
sum(combo6.duplicated())

100

In [36]:
##drop duplicates
combo7 = combo6.drop_duplicates(ignore_index = True)

In [38]:
###amount of rows
combo7.shape

(212157, 23)

In [39]:
### drop row which didn't match
combo8 = combo7[combo7['UPC'].notna()]

In [40]:
### amount of rows
combo8.shape

(212157, 23)

In [41]:
###check for duplicates rows
sum(combo8.duplicated())

0

In [42]:
#importing international catalogue data

international_catalogue = pd.read_excel(r'G:\Amazon Repatriation May 2020\UMGI Physical Catalogue_17042020.xlsx', dtype={'UPC': str})

In [44]:
### drop redundant columns
cat2 = international_catalogue[['Rep Owner','UPC']]

In [46]:
##drop duplicates
cat3 = cat2.drop_duplicates(ignore_index = True)

In [47]:
###check for duplicates rows
sum(cat3.duplicated())

0

In [49]:
##only keep first label
cat4 = cat3.drop_duplicates(subset='UPC', keep="first")

In [51]:
### drop row which didn't match
cat5 = cat4[cat4['UPC'].notna()]

In [52]:
#merging our dataset and international catalogue to return Rep Owner for each UPC

combo_cat = pd.merge(combo8,cat5[['UPC','Rep Owner']],
                     left_on='UPC', right_on=['UPC'], how='left')

In [54]:
## rename columns 
combo_cat2 = combo_cat.rename(columns={'Rep Owner': 'Rep_Owner'})

In [56]:
### count rows and columsn in combo2
combo_cat2.shape

(212157, 24)

In [57]:
###drop caroline labels
combo_cat3 = combo_cat2[~combo_cat2.Rep_Owner.str.contains('Caroline', na=False)]

In [58]:
### check row length 
combo_cat3.shape

(208416, 24)

In [59]:
###drop caroline labels
combo_cat4 = combo_cat3[~combo_cat3.Rep_Owner.str.contains('Concord', na=False)]

In [60]:
### check row length
combo_cat4.shape

(203615, 24)

In [62]:
### check row count
combo_cat4.shape

(203615, 24)

In [63]:
#importing exception list
exception_list = pd.read_excel("G:\Amazon Repatriation May 2020\exception_list.xlsx", dtype={'UPC': str})

In [64]:
#show exception list

exception_list.shape

(6166, 2)

In [66]:
##delete duplicates from exception list

exception_list.drop_duplicates(subset ="UPC", 
                     keep = False, inplace = True)

In [67]:
##show exception list
exception_list.shape

(5677, 2)

In [68]:
##only keep first label
exception_list2 = exception_list.drop_duplicates(subset='UPC', keep="first")

In [69]:
exception_list2.shape

(5677, 2)

In [70]:
#merging data with exception list to find artist exceptions

data_with_exception = pd.merge(combo_cat4,exception_list2[['UPC','Exception']],left_on="UPC", right_on=['UPC'], how='left')

In [72]:
##assigning repat percentage, changing Nan to other 50%
data_with_exception['Exception'] = data_with_exception['Exception'].fillna(.50)

In [73]:
##assigning repat percentage, classical 60%, artist exception 40%, other 50%
 
data_with_exception.loc[data_with_exception['Exception'] == .5, 'repat_%'] = .5
data_with_exception.loc[data_with_exception['Exception'] == 'Artist ICLA', 'repat_%'] = .4
data_with_exception.loc[data_with_exception['Exception'] == 'Classical', 'repat_%'] = .6

In [75]:
##change repat % column to float for equations
data_with_exception['repat_%'] = data_with_exception['repat_%'].astype(float)

In [77]:

##Repatriation equation bracket net unit price times repat% price brackets minus transfer price

data_with_exception['Repatriation'] = (data_with_exception['Net Unit Price'] * data_with_exception['repat_%']) - data_with_exception['Transfer Price']

In [79]:
##Repatriation percentage equation  bracket repatiation divded by dealer price bracket times one hundred

data_with_exception['Repatriation_Percentage_%'] = (data_with_exception['Repatriation'] / data_with_exception['Dealer Price']) *100

In [81]:
##total repatriation percentage equation stock movements times repatriation 

data_with_exception['Total_Repatriation'] = data_with_exception['unpacked_units'] * data_with_exception['Repatriation']

In [83]:
data_with_exception.shape

(203615, 29)

In [84]:
#removing any rows with negative Total_Repatriation total 

data_with_exception.drop(data_with_exception.index[data_with_exception['Total_Repatriation'] < 0], inplace = True)

In [85]:
##show data

data_with_exception.shape

(202341, 29)

In [86]:
##remove rows with any NA for Total_Repatriation

data_with_exception2 = data_with_exception[data_with_exception['Total_Repatriation'].notna()]

In [87]:
data_with_exception2.shape

(202341, 29)

In [89]:
data_with_exception2.shape

(202341, 29)

In [90]:
##remove rows with any NA for inventory owner group

data_with_exception3 = data_with_exception2[data_with_exception2['inventory_owner_group_combined'].notna()]

In [91]:
data_with_exception3.shape

(200686, 29)

In [94]:
data_with_exception3.shape

(200686, 29)

In [95]:
## rename columns for google big query upload 
data_with_exception4 = data_with_exception3.rename(columns={'Report Type': 'Report_Type', 'Transfer Price':'Transfer_Price','Dealer Price':'Dealer_Price','Net Unit Price':'Net_Unit_Price','Currency ':'Currency','repat_%':'repat_percentage','Repatriation_Percentage_%':'Repatriation_Percentage_Final'})

In [96]:
###check for duplicates rows
sum(data_with_exception4.duplicated())

0

In [97]:
##drop duplicates
data_with_exception5 = data_with_exception4.drop_duplicates(ignore_index = True)

In [98]:
data_with_exception5.shape

(200686, 29)

In [99]:
###check for duplicates rows
sum(data_with_exception5.duplicated())

0

In [114]:
### rename cost data
data_with_exception6 = data_with_exception5.rename(columns={'Currency':'Currency_for_calcuations'})

In [116]:
### duplicate currency columns
data_with_exception6['Final_Currency'] = data_with_exception6['Currency_for_calcuations']

In [122]:
data_with_exception6.loc[data_with_exception6['Currency_for_calcuations']=='GBP', ['Total_Repatriation']] *= 1.15

In [124]:
data_with_exception6['Final_Currency'] = data_with_exception6['Final_Currency'].str.replace('GBP', 'EUR')

In [126]:
data_with_exception6.to_gbq('commercial_affairs_analytics.amazon_repatriation_sat_test22_2020', 'umg-uk')

1it [00:58, 58.48s/it]
